In [2]:
import os
import xarray as xr
import cf_xarray as cfxr
import pyesgf
from pyesgf.logon import LogonManager
from pyesgf.search import SearchConnection

lm = LogonManager(verify=True)

if not lm.is_logged_on():
    myproxy_host = "esgf-data.dkrz.de"
    # if we find those in environment, use them.
    if "ESGF_USER" in os.environ and "ESGF_PASSWORD" in os.environ:
        lm.logon(
            hostname=myproxy_host,
            username=os.environ["ESGF_USER"],
            password=os.environ["ESGF_PASSWORD"],
            interactive=False,
            bootstrap=True,
        )
    else:
        lm.logon(
            hostname=myproxy_host,
            interactive=True,
            bootstrap=True,
        )

print(f"logged on: {lm.is_logged_on()}")

logged on: True


In [1]:
urls = [
    "http://esgf1.dkrz.de/thredds/dodsC/cordex-reklies/output/EUR-11/CLMcom/MIROC-MIROC5/rcp85/r1i1p1/CLMcom-CCLM4-8-17/v1/mon/tas/v20171121/tas_EUR-11_MIROC-MIROC5_rcp85_r1i1p1_CLMcom-CCLM4-8-17_v1_mon_200601-201012.nc",
    "http://esgf1.dkrz.de/thredds/dodsC/cordex-reklies/output/EUR-11/CLMcom/MIROC-MIROC5/rcp85/r1i1p1/CLMcom-CCLM4-8-17/v1/mon/tas/v20171121/tas_EUR-11_MIROC-MIROC5_rcp85_r1i1p1_CLMcom-CCLM4-8-17_v1_mon_201101-202012.nc",
    "http://esgf1.dkrz.de/thredds/dodsC/cordex-reklies/output/EUR-11/CLMcom/MIROC-MIROC5/rcp85/r1i1p1/CLMcom-CCLM4-8-17/v1/mon/tas/v20171121/tas_EUR-11_MIROC-MIROC5_rcp85_r1i1p1_CLMcom-CCLM4-8-17_v1_mon_202101-203012.nc",
    "http://esgf1.dkrz.de/thredds/dodsC/cordex-reklies/output/EUR-11/CLMcom/MIROC-MIROC5/rcp85/r1i1p1/CLMcom-CCLM4-8-17/v1/mon/tas/v20171121/tas_EUR-11_MIROC-MIROC5_rcp85_r1i1p1_CLMcom-CCLM4-8-17_v1_mon_203101-204012.nc",
    "http://esgf1.dkrz.de/thredds/dodsC/cordex-reklies/output/EUR-11/CLMcom/MIROC-MIROC5/rcp85/r1i1p1/CLMcom-CCLM4-8-17/v1/mon/tas/v20171121/tas_EUR-11_MIROC-MIROC5_rcp85_r1i1p1_CLMcom-CCLM4-8-17_v1_mon_204101-205012.nc",
    "http://esgf1.dkrz.de/thredds/dodsC/cordex-reklies/output/EUR-11/CLMcom/MIROC-MIROC5/rcp85/r1i1p1/CLMcom-CCLM4-8-17/v1/mon/tas/v20171121/tas_EUR-11_MIROC-MIROC5_rcp85_r1i1p1_CLMcom-CCLM4-8-17_v1_mon_205101-206012.nc",
    "http://esgf1.dkrz.de/thredds/dodsC/cordex-reklies/output/EUR-11/CLMcom/MIROC-MIROC5/rcp85/r1i1p1/CLMcom-CCLM4-8-17/v1/mon/tas/v20171121/tas_EUR-11_MIROC-MIROC5_rcp85_r1i1p1_CLMcom-CCLM4-8-17_v1_mon_206101-207012.nc",
    "http://esgf1.dkrz.de/thredds/dodsC/cordex-reklies/output/EUR-11/CLMcom/MIROC-MIROC5/rcp85/r1i1p1/CLMcom-CCLM4-8-17/v1/mon/tas/v20171121/tas_EUR-11_MIROC-MIROC5_rcp85_r1i1p1_CLMcom-CCLM4-8-17_v1_mon_207101-208012.nc",
    "http://esgf1.dkrz.de/thredds/dodsC/cordex-reklies/output/EUR-11/CLMcom/MIROC-MIROC5/rcp85/r1i1p1/CLMcom-CCLM4-8-17/v1/mon/tas/v20171121/tas_EUR-11_MIROC-MIROC5_rcp85_r1i1p1_CLMcom-CCLM4-8-17_v1_mon_208101-209012.nc",
    "http://esgf1.dkrz.de/thredds/dodsC/cordex-reklies/output/EUR-11/CLMcom/MIROC-MIROC5/rcp85/r1i1p1/CLMcom-CCLM4-8-17/v1/mon/tas/v20171121/tas_EUR-11_MIROC-MIROC5_rcp85_r1i1p1_CLMcom-CCLM4-8-17_v1_mon_209101-210012.nc",
]

In [4]:
ds = xr.open_mfdataset(
    urls,
    chunks={},
    parallel=False,
    combine="by_coords",
    data_vars="minimal",
    coords="minimal",
    compat="override",
)

In [5]:
ds

<xarray.Dataset>
Dimensions:       (rlon: 424, rlat: 412, time: 1140, bnds: 2, vertices: 4)
Coordinates:
  * rlon          (rlon) float64 -28.38 -28.26 -28.16 ... 17.93 18.05 18.16
  * rlat          (rlat) float64 -23.38 -23.26 -23.16 ... 21.61 21.73 21.83
  * time          (time) object 2006-01-16 12:00:00 ... 2100-12-16 12:00:00
    height        float64 ...
    lat           (rlat, rlon) float64 dask.array<chunksize=(412, 424), meta=np.ndarray>
    lon           (rlat, rlon) float64 dask.array<chunksize=(412, 424), meta=np.ndarray>
Dimensions without coordinates: bnds, vertices
Data variables:
    rotated_pole  |S64 ...
    time_bnds     (time, bnds) object dask.array<chunksize=(60, 2), meta=np.ndarray>
    lon_vertices  (rlat, rlon, vertices) float64 dask.array<chunksize=(412, 424, 4), meta=np.ndarray>
    lat_vertices  (rlat, rlon, vertices) float64 dask.array<chunksize=(412, 424, 4), meta=np.ndarray>
    tas           (time, rlat, rlon) float32 dask.array<chunksize=(60, 412, 424), meta=np.ndarray>
Attributes: (12/25)
    contact:                         klima.projektionen@dwd.de
    CORDEX_domain:                   EUR-11
    institute_id:                    CLMcom
    institution:                     CLMcom, Climate Limited-area Modelling C...
    driving_model_id:                MIROC-MIROC5
    driving_experiment_name:         rcp85
    ...                              ...
    product:                         output
    frequency:                       mon
    tracking_id:                     490ab140-e096-11e7-b22c-81c28a935756
    creation_date:                   2017-12-14T07:16:13Z
    DODS.strlen:                     0
    DODS_EXTRA.Unlimited_Dimension:  time